In [1]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from processor.processor import Processor as p
from database.adatabase import ADatabase
from indicator.indicator import Indicator
from transformer.transformer import Transformer
from time import sleep
import requests as r
from dotenv import load_dotenv
load_dotenv()
import os
import matplotlib.pyplot as plt
from time import sleep
import numpy as np
import os

In [2]:
market = ADatabase("market")
chung = ADatabase("chung")

In [3]:
market.connect()
currencies = market.retrieve("currencies")
stocks = market.retrieve("index")
market.disconnect()
crypto = False

In [12]:
if crypto == True:
    index = currencies.copy().rename(columns={"product_id":"ticker"})
    index["name"] = index["ticker"]
    index["base"] = [x.split("-")[1] for x in index["ticker"]]
    index = index[index["base"]=="USD"]
    market.connect()
    spy = market.query("crypto",{"ticker":"BTC-USD"}).rename(columns={"close":"adjclose"})[["start","adjclose","high","low","volume"]]
    spy["date"] = pd.to_datetime(spy["start"].astype(float),unit="s")
    spy["spy"] = spy["adjclose"].astype(float)
    spy["high"] = spy["high"].astype(float)
    spy["low"] = spy["low"].astype(float)
    spy["volume"] = spy["volume"].astype(float)
    market.disconnect()
else:
    index = stocks.copy()
    market.connect()
    spy = p.column_date_processing(market.query("prices",{"ticker":"JPM"}))
    spy["spy"] = spy["adjclose"]
    market.disconnect()
tickers = index["ticker"].unique()
prices = []
market.connect()
for ticker in tqdm(tickers):
    try:
        if crypto == True:
            ticker_data = market.query("crypto",{"ticker":ticker}).rename(columns={"close":"adjclose"})[["start","adjclose","high","low","volume"]]
            ticker_data["date"] = pd.to_datetime(ticker_data["start"].astype(float),unit="s")
            ticker_data["adjclose"] = ticker_data["adjclose"].astype(float)
            ticker_data["high"] = ticker_data["high"].astype(float)
            ticker_data["low"] = ticker_data["low"].astype(float)
            ticker_data["volume"] = ticker_data["volume"].astype(float)
        else:
            ticker_data = market.query("prices",{"ticker":ticker})
        if ticker_data.index.size >0:
            ticker_data["ticker"] = ticker
            price = p.column_date_processing(ticker_data)
            price = p.additional_date_cols(price)
            price = Indicator.calculate_indicators(price).copy()
            price["sell_price"] = price["adjclose"]
            prices.append(price)
    except Exception as e:
        print(str(e))
market.disconnect()

 12%|██████████████████▍                                                                                                                                    | 622/5106 [00:31<02:50, 26.31it/s]E:\projects\sapling\indicator\indicator.py:51: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price["roc"] = price[cols["price"]].pct_change(timeframe)
 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                      | 3814/5106 [03:20<01:15, 17.18it/s]E:\projects\sapling\indicator\indicator.py:51: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' t

In [13]:
sim = pd.concat(prices)

In [14]:
product = Transformer.transform(sim)
product = product[product["market_cap_rank"]>=0.7]

In [15]:
chung.connect()
chung.drop("sim")
chung.store("sim",product)
chung.disconnect()